In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

dummy_reg = DummyRegressor(strategy="mean")

Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
                 "OBSERVED_DURATION_MINUTES", "SUSPECTED_CRIME_DESCRIPTION",
                 "STOP_DURATION_MINUTES", "FIREARM_FLAG",
                 "SUSPECT_RACE_DESCRIPTION", "SUSPECT_HEIGHT",
                 "DEMEANOR_OF_PERSON_STOPPED", "SUSPECT_BODY_BUILD_TYPE"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)
Feature_test=["FIREARM_FLAG", "STOP_DURATION_MINUTES", "STOP_DURATION_MINUTES"]
#C_Feature_test=["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION","SUSPECT_RACE_DESCRIPTION","DEMEANOR_OF_PERSON_STOPPED"]



Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', float('NaN'), inplace=True)


lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])

Test1["SUSPECT_ARRESTED_FLAG"].fillna(Test1["SUSPECT_ARRESTED_FLAG"].median,inplace=True)
Test1["FIREARM_FLAG"].fillna(Test1["FIREARM_FLAG"].median,inplace=True)



Test1.to_csv('output.csv', index=False)
#one Hot encoding
OneHot_Encode_feature = ["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION","SUSPECT_RACE_DESCRIPTION"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[OneHot_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(OneHot_Encode_feature))
#Label_df.fillna(value=0, inplace=True)

#label encoding
LE=LabelEncoder()
Test1['SUSPECT_BODY_BUILD_TYPE']=LE.fit_transform(Test1["SUSPECT_BODY_BUILD_TYPE"])
Test1['SUSPECT_BODY_BUILD_TYPE'].fillna(Test1['SUSPECT_BODY_BUILD_TYPE'].mean, inplace=True)

cat_feature=list(Label_df.columns)+['SUSPECT_BODY_BUILD_TYPE']
#with everything, un comment the below code to include both Categorical data and numeric in RandomForest
#cat_feature=list(Label_df.columns)+['SUSPECT_BODY_BUILD_TYPE']+Feature_test


Test1 = pd.concat([Test1, Label_df], axis=1)

# drop the original categorical feature column
Test1 = Test1.drop(columns=OneHot_Encode_feature)



train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=102)


# extract the categorical features and target variable from the training and testing data
#Random Forest
X_train = train_data[cat_feature] # training features_for Random forest
y_train = train_data['SUSPECT_ARRESTED_FLAG'] # training target variable for random forest

X_test = test_data[cat_feature] # testing features
y_test = test_data['SUSPECT_ARRESTED_FLAG'] # testing target variable

#Linear regression
X_train_linear=train_data[Feature_test] 
y_train_linear=train_data["SUSPECT_ARRESTED_FLAG"]

X_test_linear=test_data[Feature_test]
y_test_linear=test_data['SUSPECT_ARRESTED_FLAG']



In [13]:
Model_N=MultinomialNB()
Model_N.fit(X_train,y_train)
N_prediction=Model_N.predict(X_test)
train_N_prediction=Model_N.predict(X_train)
N_train_accuracy=accuracy_score(y_train,train_N_prediction)
N_accuracy=accuracy_score(y_test,N_prediction)

print('Train_Accuracy:', N_accuracy)
print('Accuracy:', N_train_accuracy)



Train_Accuracy: 0.6964581264481959
Accuracy: 0.685704825759457


In [14]:



model_R = RandomForestClassifier(random_state=42)
#model_R.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])

#with only caterogical data but transform to numeric/ with both

model_R.fit(X_train,y_train)
Prediction=model_R.predict(X_test)

train_Prediction =model_R.predict(X_train)


train_accuracy=accuracy_score(y_train,train_Prediction)

accuracy = accuracy_score(y_test, Prediction)


print('Train_Accuracy:', train_accuracy)
print('Accuracy:', accuracy)


Train_Accuracy: 0.7208012581739922
Accuracy: 0.6881827209533267


In [15]:
model = LinearRegression()
model.fit(X_train_linear, y_train_linear)

# make predictions on the testing and training data, and calculate RMSE
y_test_pred = model.predict(X_test_linear)
y_train_pred = model.predict(X_train_linear)
test_rmse = mean_squared_error(y_test_linear, y_test_pred, squared=False)
train_rmse = mean_squared_error(y_train_linear, y_train_pred, squared=False)

print('Testing RMSE:', test_rmse)
print('Training RMSE:', train_rmse)

Testing RMSE: 0.4374610649498002
Training RMSE: 0.43887036091262194
